<a href="https://colab.research.google.com/github/rawar/tensorflow-notebooks/blob/master/lstm_text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Textgenerator based on news

Based on Enriques work of an [Word-level LSTM text generator](https://medium.com/coinmonks/word-level-lstm-text-generator-creating-automatic-song-lyrics-with-neural-networks-b8a1617104fb)

In [1]:
!pip install tf-nightly-gpu

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [0]:
url = 'https://gist.githubusercontent.com/rawar/ae4cce269e29c2826163fbca60b544f4/raw/20594a832b44de4b4a5160fb2732399ba09a70e9/reco-content-data.csv'
text_corpus = pd.read_csv(url)

In [4]:
headlines_data_frame = text_corpus['title']
headlines_data_frame = headlines_data_frame.dropna()
headlines_data_frame.head()

0                             Google verkauft Motorola
1    Touchscreen im Winter: Handy-Handschuhe selber...
2    \Bei WhatsApp als Kontakt blockiert: So merkt ...
3    Paper: Facebook stellt neue App im Flipboard-S...
4               WhatsApp: mehr Privatsphäre für Nutzer
Name: title, dtype: object

In [6]:
text_in_words = headlines_data_frame.str.split(' ').tolist()
print('Number of sentences:', len(text_in_words))

Number of sentences: 15751


In [7]:
print(text_in_words[15749:15750])

[['Fatburner-Training:', 'Diese', 'Übungen', 'sind', 'für', 'Männer']]


In [27]:
MAX_WORDS_PER_HEADLINE = 5
for example in text_in_words:
  new_max = len(example)
  if new_max > MAX_WORDS_PER_HEADLINE:
    MAX_WORDS_PER_HEADLINE = new_max
    
print(MAX_WORDS_PER_HEADLINE)

16


In [0]:
words = set()
for sentence in text_in_words:
  for word in sentence:
    words.add(word)

In [10]:
len(words)

10654

In [0]:
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

In [0]:
SEQUENCE_LEN = 2
MIN_WORD_FREQUENCY = 1
STEP = 1
BATCH_SIZE = 32
EPOCHS = 50

In [0]:
sentences = []
next_words = []
ignored = 0
for sentence in text_in_words:
  for i in range(0, len(sentence) - SEQUENCE_LEN, STEP):
    sentences.append(sentence[i:i + SEQUENCE_LEN])
    next_words.append(sentence[i + SEQUENCE_LEN])

In [0]:
#print(sentences[0])

In [0]:
#print(next_words[0])

In [0]:
#print(sentences[2])

In [0]:
#print(next_words[2])

In [0]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)

In [19]:
(sentences_train, next_words_train), (sentences_test, next_words_test) = shuffle_and_split_training_set(sentences, next_words)

Shuffling sentences
Size of training set = 98214
Size of test set = 2005


In [20]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(words), output_dim=1024),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(words)),
    tf.keras.layers.Activation('softmax')
])

W0512 16:55:54.995031 139959589132160 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0512 16:55:54.998946 139959589132160 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0512 16:55:55.007711 139959589132160 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init

In [0]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [23]:
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [0]:
file_path = "./checkpoints/lstm-text-gen-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
                "loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % \
                (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)

    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(MAX_WORDS_PER_HEADLINE):
            x_pred = np.zeros((1, SEQUENCE_LEN))
            for t, word in enumerate(sentence):
                x_pred[0, t] = word_indices[word]

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()


In [0]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=20)
callbacks_list = [checkpoint, print_callback, early_stopping]

In [0]:
examples_file = open('examples.txt', "w")

In [32]:
history = model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                    epochs=EPOCHS,
                    callbacks=callbacks_list,
                    validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                    validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

Epoch 1/50
3132/3132 [==============================] - 44s 14ms/step - loss: 0.9769 - acc: 0.8010 - val_loss: 0.8297 - val_acc: 0.8224
Epoch 2/50
3132/3132 [==============================] - 45s 14ms/step - loss: 0.8271 - acc: 0.8195 - val_loss: 0.6810 - val_acc: 0.8472
Epoch 3/50
3132/3132 [==============================] - 45s 14ms/step - loss: 0.7276 - acc: 0.8333 - val_loss: 0.5945 - val_acc: 0.8591
Epoch 4/50
3132/3132 [==============================] - 45s 14ms/step - loss: 0.6726 - acc: 0.8410 - val_loss: 0.5360 - val_acc: 0.8681
Epoch 5/50
3132/3132 [==============================] - 46s 15ms/step - loss: 0.6427 - acc: 0.8456 - val_loss: 0.5271 - val_acc: 0.8686
Epoch 6/50
3132/3132 [==============================] - 44s 14ms/step - loss: 0.6192 - acc: 0.8491 - val_loss: 0.4922 - val_acc: 0.8780
Epoch 7/50
3132/3132 [==============================] - 45s 14ms/step - loss: 0.6060 - acc: 0.8499 - val_loss: 0.4728 - val_acc: 0.8745
Epoch 8/50
3132/3132 [==========================

In [36]:
!tail -n 25 examples.txt

ein TIE-Fighter des Autoherstellers aus" aus" für Android und iOS 10: So setzt Ihr die Anzeige-Modi Watch im

----- Generating text after Epoch: 34
----- Diversity:0.3
----- Generating with seed:
"– und"
– und mehr Kraft aus?\"" für Windows Phone 8 und X: Akku lässt sich nicht als Bluetooth-Lautsprecher nutzen
----- Diversity:0.4
----- Generating with seed:
"– und"
– und mehr Kraft aus?\"" für Android und iOS erschienen\"" Beta für Android und iOS erschienen\"" der neuen
----- Diversity:0.5
----- Generating with seed:
"– und"
– und mehr Kraft aus?\"" für Android und iOS erschienen\"" Apple Watch Series 4 soll mit größerem Bildschirm
----- Diversity:0.6
----- Generating with seed:
"– und"
– und mehr Kraft aus?\"" für Android und Windows verfolgen und Co. für Weihnachten ein Pikachu mit X-Mas-Mütze
----- Diversity:0.7
----- Generating with seed:
"– und"
– und mehr Kraft aus?\"" für Windows Phone 8 aufgetaucht sein noch mehr Apple-Apps für Android und iOS


In [37]:
!ls -l checkpoints

total 4345504
-rw-r--r-- 1 root root 177988152 May 12 17:00 lstm-text-gen-epoch001-words10654-sequence2-minfreq1-loss0.9769-acc0.8010-val_loss0.8297-val_acc0.8224
-rw-r--r-- 1 root root 177988152 May 12 15:45 lstm-text-gen-epoch001-words10654-sequence2-minfreq1-loss1.9330-acc0.6770-val_loss1.5170-val_acc0.7207
-rw-r--r-- 1 root root 177988152 May 12 15:43 lstm-text-gen-epoch001-words10654-sequence2-minfreq1-loss2.6888-acc0.6056-val_loss2.5850-val_acc0.6205
-rw-r--r-- 1 root root 177988152 May 12 16:57 lstm-text-gen-epoch001-words10654-sequence2-minfreq1-loss3.4357-acc0.5530-val_loss4.0911-val_acc0.5670
-rw-r--r-- 1 root root 177988152 May 12 15:41 lstm-text-gen-epoch001-words10654-sequence2-minfreq1-loss3.4432-acc0.5528-val_loss4.0844-val_acc0.5719
-rw-r--r-- 1 root root 177988152 May 12 17:01 lstm-text-gen-epoch002-words10654-sequence2-minfreq1-loss0.8271-acc0.8195-val_loss0.6810-val_acc0.8472
-rw-r--r-- 1 root root 177988152 May 12 15:45 lstm-text-gen-epoch002-words10654-sequence2-mi

In [0]:
from google.colab import files
model_json = model.to_json()
with open("lstm-text-gen_acc88.json", "w") as json_file:
    json_file.write(model_json)

In [0]:
files.download("lstm-text-gen_acc88.json")

In [0]:
model.save_weights("lstm-text-gen_model.h5")
print("Saved model to disk")

In [41]:
!ls -lh

total 57M
drwxr-xr-x 2 root root 4.0K May 12 17:10 checkpoints
-rw-r--r-- 1 root root  39K May 12 17:26 examples.txt
-rw-r--r-- 1 root root 2.3K May 12 17:29 lstm-text-gen_acc88.json
-rw-r--r-- 1 root root  57M May 12 17:29 lstm-text-gen_model.h5
drwxr-xr-x 1 root root 4.0K May  8 16:22 sample_data


In [42]:
files.download('lstm-text-gen_model.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 39134, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
files.download("examples.txt")

In [0]:
def generate_text(model, indices_word, word_indices, seed,
                  sequence_length, diversity, quantity):
    """
    Similar to lstm_train::on_epoch_end
    Used to generate text using a trained model
    :param model: the trained Keras model (with model.load)
    :param indices_word: a dictionary pointing to the words
    :param seed: a string to be used as seed (already validated and padded)
    :param sequence_length: how many words are given to the model to generate
    :param diversity: is the "temperature" of the sample function (usually between 0.1 and 2)
    :param quantity: quantity of words to generate
    :return: Nothing, for now only writes the text to console
    """
    sentence = seed.split(" ")
    print("----- Generating text")
    print('----- Diversity:' + str(diversity))
    print('----- Generating with seed:"' + seed +'"')

    print(seed)
    for i in range(quantity):
        x_pred = np.zeros((1, sequence_length))
        for t, word in enumerate(sentence):
            x_pred[0, t] = word_indices[word]

        preds = model.predict(x_pred, verbose=0)[0]
   
        next_index = sample(preds, diversity)
        next_word = indices_word[next_index]

        sentence = sentence[1:]
        sentence.append(next_word)

        print(" "+next_word, end="")



In [77]:
generate_text(model, indices_word, word_indices, "Neues Samsung", 2, 0.5, 10)

----- Generating text
----- Diversity:0.5
----- Generating with seed:"Neues Samsung"
Neues Samsung
 stellt 6-GB-RAM-Chip vor dem Start: Das kann das kleine P8